In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy.special import gamma
from itertools import combinations

sns.set_style('white')

In [ ]:
def pos_to_distance(pos: np.ndarray) -> np.ndarray:
    return np.array([np.linalg.norm(c1 - c2) for c1, c2 in combinations(pos, 2)])

def mb(sigma: float, size: tuple) -> np.ndarray:
    return np.sqrt(np.sum([np.square(np.random.normal(0, sigma, size=size)) for _ in range(3)], axis=0))

def mb_pdf(x: np.ndarray, a: float) -> np.ndarray:
    return np.where(x > 0,
        np.sqrt(2 / np.pi) * np.power(x, 2) * np.exp(- np.power(x, 2) / (2 * np.power(a, 2))) / np.power(a, 3),
        0)

def normal_pdf(x: np.ndarray, sigma: float) -> np.ndarray:
    return np.exp(- np.square(x / sigma) / 2) / (sigma * np.sqrt(2 * np.pi))

In [ ]:
COLORS = [sns.xkcd_rgb['denim blue'], 'orange', sns.xkcd_rgb['medium green']]

In [ ]:
n_molecule = 20
D = 1
sigmas = [0.002, 0.2, 1, 5, 10, 50]
d_distr = {}
for sigma in sigmas:
    d_distance = []
    for _ in tqdm(range(int(5e5))):
        pos = np.array([[0, 0, 0], [D, 0, 0]])
        pos_ = pos + np.random.normal(0, sigma, size=pos.shape)
        d_distance.append(pos_to_distance(pos_) - pos_to_distance(pos))
    d_distr[sigma] = np.stack(d_distance).reshape(-1)

In [ ]:
def p(x: np.ndarray, sigma: float) -> np.ndarray:
    k = 2 / gamma((3 - 2 * np.exp(- sigma)) / 2)
    return k * np.power(x + D, 2 * np.power(1 - np.exp(- sigma / D), np.sqrt(np.pi) / 4)) * np.exp(- np.square(x) / 4 / np.square(sigma))

def score_p(x: np.ndarray, sigma: float) -> np.ndarray:
    return (1 - np.exp(-np.sqrt(sigma / D))) * (2 * sigma / (x + D)) - x / (2 * sigma)

def score_Gaussion(x: np.ndarray, sigma: float) -> np.ndarray:
    return - x / sigma

In [ ]:
fig = plt.figure(figsize=(80, 10), dpi=500)
sns.set(context='notebook', style='ticks', font_scale=3)
alpha = 0.85
linewidth = 8
for i, (sigma, dis) in enumerate(d_distr.items()):
    ax1 = fig.add_subplot(1, 6, i+1)
    dis = np.sort(dis)
    sns.kdeplot(data=dis, label='true distribution', color=COLORS[1], legend=True, ax=ax1, linewidth=linewidth, alpha=alpha)
    # plt.legend()
    plt.ylim(bottom=0)

    pdf = p(dis, sigma)
    ax2 = ax1.twinx()
    sns.lineplot(x=dis, y=pdf, color=COLORS[0], linewidth=linewidth, linestyle=':')
    plt.ylim(bottom=0)
    plt.yticks([])
    
    plt.title(f'$\sigma$ = {sigma}', fontsize=80)
    plt.xticks(fontsize=40)
    plt.yticks(fontsize=40)
plt.savefig('score-approx-pdf.pdf')